In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/workspace/GTFS')

from src.timeseries_processing import DataSplitter, DataStandardizer, DataSeparater
from src.model_training import DelayRegressionModel

pd.set_option('display.max_columns', None)

2025-09-18 03:05:05.402406: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-18 03:05:05.423578: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 03:05:06.506735: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-18 03:05:10.895366: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [2]:
delay_features = pd.read_csv('/workspace/GTFS/merged_dataset.csv')

In [3]:
data_separater = DataSeparater()
feature_cols = ['weather', 'temp', 'precipitation', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'delay_deviation']
target_col = 'arrival_delay'

X_delay, y_delay, used_features = data_separater.X_Y_separate(delay_features, target_col, feature_cols)
print(f"シーケンス形状: X={X_delay.shape}, y={y_delay.shape}")

Using features: ['weather', 'temp', 'precipitation', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'delay_deviation']
シーケンス形状: X=(3446, 8), y=(3446,)


In [4]:
# データ分割
data_splitter = DataSplitter()
X_delay_train, X_delay_test, y_delay_train, y_delay_test = data_splitter.train_test_split_temporal(
    X_delay, y_delay, train_ratio=0.9
)

print(f"\n=== Delay Prediction Data Split Results ===")
print(f"  Training data: X={X_delay_train.shape}, y={y_delay_train.shape}")
print(f"  Test data: X={X_delay_test.shape}, y={y_delay_test.shape}")

# データ標準化
standardizer = DataStandardizer()
X_delay_train_scaled = standardizer.fit_transform_features(X_delay_train)
X_delay_test_scaled = standardizer.transform_features(X_delay_test)
y_delay_train_scaled = standardizer.fit_transform_targets(y_delay_train)
y_delay_test_scaled = standardizer.transform_targets(y_delay_test)


=== Delay Prediction Data Split Results ===
  Training data: X=(3101, 8), y=(3101,)
  Test data: X=(345, 8), y=(345,)


In [ ]:
# モデル作成（Ridge回帰）
model = DelayRegressionModel(model_type='ridge', normalize_features=True)

# 訓練
history = model.train_model(X_delay_train_scaled, y_delay_train_scaled, validation_split=0.2)

# 予測
predictions = model.predict(X_delay_test_scaled)

# ハイパーパラメータチューニング
tuning_results = model.hyperparameter_tuning(X_delay_train_scaled, y_delay_train_scaled)

# モデル保存
# model.save_model('delay_model.pkl')